# import packages

In [1]:
# import pandas package
# import os package

import pandas as pd

In [2]:
import os

# Mount the drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load the dataset

In [49]:
# load the credential_leak_demo.db into your google drive  - no code needed to do this step

# specify the directory path where the dataset is located
DIR = '/content/drive/MyDrive/Colab Notebooks/data'
datafile = 'credential_leak_demo_object.db'
filename = DIR+'/'+datafile

In [42]:
os.chdir(DIR)
!pwd

/content/drive/MyDrive/Colab Notebooks/data


In [43]:
# check the file is present or not
!ls

'alerts (1).json'		  eml		        superstore.csv
'alerts (2).json'		  incident_notes.txt    superstore.dbf
 alerts.json			  ioc_feed.xml	        superstore_json.json
'auth_logs (1).csv'		  my_database.db        superstore_tsv.tsv
 auth_logs.csv			  netflow.jsonl         superstore.txt
 credential_leak_demo.db	  security_policy.yml   superstore_xls.xlsx
 credential_leak_demo_object.db   soc_alerts.xlsx       web_access.log
 dns_queries.tsv		  soc_dump.sql


In [50]:
# “Open (or create) the SQLite database file and keep the connection in conn
import sqlite3
conn = sqlite3.connect(filename)

Without a connection:

    Python cannot read tables
    Python cannot execute SQL queries
    Python cannot convert tables to DataFrames
    All database operations go through conn.

In [40]:
!pwd

/content


In [51]:
# To explore the database
tables = pd.read_sql_query(
    "SELECT * FROM sqlite_master", conn
)
tables

,type,name,tbl_name,rootpage,sql
0,table,users,users,2,CREATE TABLE users (\n user_id TEXT PRIMARY K...
1,index,sqlite_autoindex_users_1,users,3,None
2,table,password_resets,password_resets,4,CREATE TABLE password_resets (\n reset_id TEX...
3,index,sqlite_autoindex_password_resets_1,password_resets,5,None
4,table,sessions,sessions,6,CREATE TABLE sessions (\n session_id TEXT PRI...
5,index,sqlite_autoindex_sessions_1,sessions,7,None
6,table,api_keys,api_keys,8,CREATE TABLE api_keys (\n key_id TEXT PRIMARY...
7,index,sqlite_autoindex_api_keys_1,api_keys,9,None
8,table,audit_log,audit_log,10,CREATE TABLE audit_log (\n event_id TEXT PRIM...
9,index,sqlite_autoindex_audit_log_1,audit_log,11,None


In [52]:
# Improvised
# To check all the tables in the database
tables = pd.read_sql_query(
    "SELECT * FROM sqlite_master WHERE type='table'", conn
)
tables

,type,name,tbl_name,rootpage,sql
0,table,users,users,2,CREATE TABLE users (\n user_id TEXT PRIMARY K...
1,table,password_resets,password_resets,4,CREATE TABLE password_resets (\n reset_id TEX...
2,table,sessions,sessions,6,CREATE TABLE sessions (\n session_id TEXT PRI...
3,table,api_keys,api_keys,8,CREATE TABLE api_keys (\n key_id TEXT PRIMARY...
4,table,audit_log,audit_log,10,CREATE TABLE audit_log (\n event_id TEXT PRIM...


In [54]:
# Improvised
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;",
    conn
)["name"].tolist()
tables

['api_keys', 'audit_log', 'password_resets', 'sessions', 'users']

  SQLite stores tables and indexes as B-trees:
      
      Root page → internal pages → leaf pages
      Efficient searching, insertion, deletion  
      rootpage tells SQLite where to start traversal.

# Reading from the files

In [56]:
import sqlite3
conn = sqlite3.connect(filename)
df= pd.read_sql_query("SELECT * FROM users;", conn)
df

,user_id,username,email,password_hash,password_salt,role,mfa_enabled,last_login_utc,created_utc
0,1,user00001,user00001@example.com,8d914e1cbb60b890c7399f2910cec96f8a5a8271363223...,bVrpoiVgRV5IfLBc,student,1,2026-01-23T15:16:00Z,2026-01-23T15:16:00Z
1,2,user00002,user00002@corp.local,581e669218963490d73113876dde788b242cedd3bf6cfe...,SBvrjn9Wvgfygw2w,student,0,2026-02-16T22:00:00Z,2026-01-02T22:00:00Z
2,3,user00003,user00003@example.com,53a95abbb5fc355643afa051e051096dcccbcdad3332c5...,cQoXsf2o3gyrDO1x,student,0,2026-01-16T06:47:00Z,2025-12-23T06:47:00Z
3,4,user00004,user00004@mail.test,8330d1676585fc69ec535bed89b7975db8b0a23501a8ff...,7OSJoRu1XXdo0cZu,student,0,2026-02-13T12:19:00Z,2026-01-28T12:19:00Z
4,5,user00005,user00005@corp.local,f2889184f365ce51dd86bcdeaa4b03f74cd938666af404...,pVJIqVLB5LzxoiGF,student,0,2026-02-16T14:28:00Z,2026-01-12T14:28:00Z
...,...,...,...,...,...,...,...,...,...
5995,5996,user05996,user05996@example.com,b5eeefc9a6aa2ba2126f999db37b163ecd93919620f3f1...,1oU49MRWypWAO7hb,student,0,None,2025-12-20T06:43:00Z
5996,5997,user05997,user05997@corp.local,7478fedc0c6703ab0313b84a47a1cee2a4dcbd89e595fe...,OnV3joXTYF6g8JuB,student,0,2026-02-11T04:47:00Z,2026-01-29T04:47:00Z
5997,5998,user05998,user05998@service.io,e6ca07d767ad01a8ab27a52cbaf04e85a7e31f39db9367...,wbL15YRXMMA0sj8H,student,1,2026-02-07T19:15:00Z,2026-01-23T19:15:00Z
5998,5999,user05999,user05999@corp.local,1b92217c412b25b5a6443ff47c70dc43c3daf94b2918ec...,Jf0ZzGdH9pv6uTQE,student,1,2026-01-19T19:58:00Z,2025-12-20T19:58:00Z


In [60]:
# Task:
# To check df is empty or not
df.empty

False

In [61]:
#To check the dimension
df.ndim

2

In [62]:
# Task:
# find the datastructure of df
type(df)

pandas.core.frame.DataFrame

In [64]:
# Task:
# check the dataframe
df

,user_id,username,email,password_hash,password_salt,role,mfa_enabled,last_login_utc,created_utc
0,1,user00001,user00001@example.com,8d914e1cbb60b890c7399f2910cec96f8a5a8271363223...,bVrpoiVgRV5IfLBc,student,1,2026-01-23T15:16:00Z,2026-01-23T15:16:00Z
1,2,user00002,user00002@corp.local,581e669218963490d73113876dde788b242cedd3bf6cfe...,SBvrjn9Wvgfygw2w,student,0,2026-02-16T22:00:00Z,2026-01-02T22:00:00Z
2,3,user00003,user00003@example.com,53a95abbb5fc355643afa051e051096dcccbcdad3332c5...,cQoXsf2o3gyrDO1x,student,0,2026-01-16T06:47:00Z,2025-12-23T06:47:00Z
3,4,user00004,user00004@mail.test,8330d1676585fc69ec535bed89b7975db8b0a23501a8ff...,7OSJoRu1XXdo0cZu,student,0,2026-02-13T12:19:00Z,2026-01-28T12:19:00Z
4,5,user00005,user00005@corp.local,f2889184f365ce51dd86bcdeaa4b03f74cd938666af404...,pVJIqVLB5LzxoiGF,student,0,2026-02-16T14:28:00Z,2026-01-12T14:28:00Z
...,...,...,...,...,...,...,...,...,...
5995,5996,user05996,user05996@example.com,b5eeefc9a6aa2ba2126f999db37b163ecd93919620f3f1...,1oU49MRWypWAO7hb,student,0,None,2025-12-20T06:43:00Z
5996,5997,user05997,user05997@corp.local,7478fedc0c6703ab0313b84a47a1cee2a4dcbd89e595fe...,OnV3joXTYF6g8JuB,student,0,2026-02-11T04:47:00Z,2026-01-29T04:47:00Z
5997,5998,user05998,user05998@service.io,e6ca07d767ad01a8ab27a52cbaf04e85a7e31f39db9367...,wbL15YRXMMA0sj8H,student,1,2026-02-07T19:15:00Z,2026-01-23T19:15:00Z
5998,5999,user05999,user05999@corp.local,1b92217c412b25b5a6443ff47c70dc43c3daf94b2918ec...,Jf0ZzGdH9pv6uTQE,student,1,2026-01-19T19:58:00Z,2025-12-20T19:58:00Z


In [65]:
# Task:
# Find the size
df.size

54000

In [66]:
# Task:
# Find the total number of rows and columns
df.shape

(6000, 9)

In [67]:
# Task:
# Find the number of records
df.shape[0]

6000

In [68]:
# Task:
# Find the total number of features / columns
df.shape[1]

9

In [70]:
#Task:
# find the summary of the dataframe ie. column name + data type + non-null count
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         6000 non-null   object
 1   username        6000 non-null   object
 2   email           6000 non-null   object
 3   password_hash   6000 non-null   object
 4   password_salt   6000 non-null   object
 5   role            6000 non-null   object
 6   mfa_enabled     6000 non-null   object
 7   last_login_utc  5369 non-null   object
 8   created_utc     6000 non-null   object
dtypes: object(9)
memory usage: 422.0+ KB


In [72]:

#Find summary statistics
df.describe()

,user_id,username,email,password_hash,password_salt,role,mfa_enabled,last_login_utc,created_utc
count,6000,6000,6000,6000,6000,6000,6000,5369,6000
unique,6000,6000,6000,6000,6000,4,2,5269,5783
top,5984,user05984,user05984@service.io,b20ac7ef38dbfefc99805da85a81478863ef8c39ce0199...,mav4wGE9zJifpmHn,student,0,2026-01-29T18:18:00Z,2025-12-10T03:41:00Z
freq,1,1,1,1,1,4223,3879,3,3


In [73]:
# find the datatypes
df.dtypes

,0
user_id,object
username,object
email,object
password_hash,object
password_salt,object
role,object
mfa_enabled,object
last_login_utc,object
created_utc,object


In [82]:
#generate a summary of descriptive statistics for non-numeric
df.describe()

,user_id,username,email,password_hash,password_salt,role,mfa_enabled,last_login_utc,created_utc
count,6000,6000,6000,6000,6000,6000,6000,5369,6000
unique,6000,6000,6000,6000,6000,4,2,5269,5783
top,5984,user05984,user05984@service.io,b20ac7ef38dbfefc99805da85a81478863ef8c39ce0199...,mav4wGE9zJifpmHn,student,0,2026-01-29T18:18:00Z,2025-12-10T03:41:00Z
freq,1,1,1,1,1,4223,3879,3,3


In [83]:
#Task:
#check is the any null values
df.isna()

,user_id,username,email,password_hash,password_salt,role,mfa_enabled,last_login_utc,created_utc
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
5995,False,False,False,False,False,False,False,True,False
5996,False,False,False,False,False,False,False,False,False
5997,False,False,False,False,False,False,False,False,False
5998,False,False,False,False,False,False,False,False,False


In [84]:
# Task:
# Check not null values
df.notna()

,user_id,username,email,password_hash,password_salt,role,mfa_enabled,last_login_utc,created_utc
0,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...
5995,True,True,True,True,True,True,True,False,True
5996,True,True,True,True,True,True,True,True,True
5997,True,True,True,True,True,True,True,True,True
5998,True,True,True,True,True,True,True,True,True


In [85]:
# Task:
#Total non-null counts for each column
df.notna().sum()

,0
user_id,6000
username,6000
email,6000
password_hash,6000
password_salt,6000
role,6000
mfa_enabled,6000
last_login_utc,5369
created_utc,6000


In [88]:
#Which columns contain missing values
df.isna().any()

,0
user_id,False
username,False
email,False
password_hash,False
password_salt,False
role,False
mfa_enabled,False
last_login_utc,True
created_utc,False


In [89]:
# Task:
# Count missing values
df.isna().sum()

,0
user_id,0
username,0
email,0
password_hash,0
password_salt,0
role,0
mfa_enabled,0
last_login_utc,631
created_utc,0


In [80]:
# Task:
# Find the first 5 records
df.head()

,user_id,username,email,password_hash,password_salt,role,mfa_enabled,last_login_utc,created_utc
0,1,user00001,user00001@example.com,8d914e1cbb60b890c7399f2910cec96f8a5a8271363223...,bVrpoiVgRV5IfLBc,student,1,2026-01-23T15:16:00Z,2026-01-23T15:16:00Z
1,2,user00002,user00002@corp.local,581e669218963490d73113876dde788b242cedd3bf6cfe...,SBvrjn9Wvgfygw2w,student,0,2026-02-16T22:00:00Z,2026-01-02T22:00:00Z
2,3,user00003,user00003@example.com,53a95abbb5fc355643afa051e051096dcccbcdad3332c5...,cQoXsf2o3gyrDO1x,student,0,2026-01-16T06:47:00Z,2025-12-23T06:47:00Z
3,4,user00004,user00004@mail.test,8330d1676585fc69ec535bed89b7975db8b0a23501a8ff...,7OSJoRu1XXdo0cZu,student,0,2026-02-13T12:19:00Z,2026-01-28T12:19:00Z
4,5,user00005,user00005@corp.local,f2889184f365ce51dd86bcdeaa4b03f74cd938666af404...,pVJIqVLB5LzxoiGF,student,0,2026-02-16T14:28:00Z,2026-01-12T14:28:00Z


In [90]:
# Task:
# Find the first 10 records
df.head(10)

,user_id,username,email,password_hash,password_salt,role,mfa_enabled,last_login_utc,created_utc
0,1,user00001,user00001@example.com,8d914e1cbb60b890c7399f2910cec96f8a5a8271363223...,bVrpoiVgRV5IfLBc,student,1,2026-01-23T15:16:00Z,2026-01-23T15:16:00Z
1,2,user00002,user00002@corp.local,581e669218963490d73113876dde788b242cedd3bf6cfe...,SBvrjn9Wvgfygw2w,student,0,2026-02-16T22:00:00Z,2026-01-02T22:00:00Z
2,3,user00003,user00003@example.com,53a95abbb5fc355643afa051e051096dcccbcdad3332c5...,cQoXsf2o3gyrDO1x,student,0,2026-01-16T06:47:00Z,2025-12-23T06:47:00Z
3,4,user00004,user00004@mail.test,8330d1676585fc69ec535bed89b7975db8b0a23501a8ff...,7OSJoRu1XXdo0cZu,student,0,2026-02-13T12:19:00Z,2026-01-28T12:19:00Z
4,5,user00005,user00005@corp.local,f2889184f365ce51dd86bcdeaa4b03f74cd938666af404...,pVJIqVLB5LzxoiGF,student,0,2026-02-16T14:28:00Z,2026-01-12T14:28:00Z
5,6,user00006,user00006@example.com,8fc5e174e1ba391a10e72f60c2ed6c9fc4d8b28b8c6800...,RUhR4IWrXPvhsBkD,staff,1,2026-02-09T03:48:00Z,2026-01-28T03:48:00Z
6,7,user00007,user00007@corp.local,5ffe482fbdddeef2a9c45ca7e97c6918a5390ea1154fa6...,xWkI9X7H6aMuFbh7,student,0,None,2025-12-07T07:11:00Z
7,8,user00008,user00008@service.io,44cef8e1ebedd1ced27557fd814c5cfd14d996b367273b...,XiiQE8JkqH3MB9n7,staff,0,2026-01-13T13:33:00Z,2025-12-23T13:33:00Z
8,9,user00009,user00009@example.com,d647ee8645b5faf2aea068260be22c0a4559d90980e118...,LJoLoaeTOdoe5c3v,student,0,2026-01-25T01:13:00Z,2026-01-13T01:13:00Z
9,10,user00010,user00010@example.com,85ad4d23a799be4ac733105fa0777019d5cce3dbaa403a...,gQBwBAD3UdRPPgdz,student,0,2026-01-11T20:40:00Z,2025-12-07T20:40:00Z


In [91]:
# Task:
# Find the last 15 records
df.tail(15)

,user_id,username,email,password_hash,password_salt,role,mfa_enabled,last_login_utc,created_utc
5985,5986,user05986,user05986@corp.local,166b9a72179a3e8d7a651274910c472cf570f9cad2ef00...,UPsUsHJSpF7wT0fX,student,0,2026-02-08T17:20:00Z,2026-01-18T17:20:00Z
5986,5987,user05987,user05987@service.io,faf9b431b557b860e38e72fd20932b9ba95f441c691ed8...,EY3QQcqFsDhYQitR,student,1,2026-01-14T00:09:00Z,2026-01-14T00:09:00Z
5987,5988,user05988,user05988@corp.local,b64c1135e7c89576a8a78e6d1474e5c00af32f26f322be...,eNcAHjkaCw8XGssW,staff,1,None,2026-01-01T12:18:00Z
5988,5989,user05989,user05989@corp.local,4ce06918948c8c017609502fe187a50374796ad3e28d49...,t44KM8PT2znavuyP,staff,1,None,2026-01-06T08:58:00Z
5989,5990,user05990,user05990@mail.test,218a97c330365c53e365fd9317a6c27e246c0031582cb2...,ZLcx5jXqswBjGoEp,student,0,2026-02-13T06:01:00Z,2025-12-25T06:01:00Z
5990,5991,user05991,user05991@example.com,373ee42c1d25579da009006d5cb26474f4d070d25b545d...,ypU8tz9OVmMKHUOh,student,0,2026-02-22T16:44:00Z,2026-01-23T16:44:00Z
5991,5992,user05992,user05992@corp.local,5df2500f1ad01293301895dd26174c3f57d1cd03c81120...,e33rXnzuV7dpxa2s,student,0,2026-02-09T19:58:00Z,2026-01-07T19:58:00Z
5992,5993,user05993,user05993@example.com,69a441e681264a10318f46518b504c122540652f51822c...,6DnEvqDBZPXbRoIo,student,1,2026-03-01T05:51:00Z,2026-01-23T05:51:00Z
5993,5994,user05994,user05994@corp.local,228073cab25cb05d2177a26b59856ba91539c6bda64754...,IxYoPicJgifpf2w8,student,1,2025-12-16T11:57:00Z,2025-12-09T11:57:00Z
5994,5995,user05995,user05995@mail.test,b2ff31419a2969d041931e1d18992093ac4d4260b78f14...,fBDMbGSQux6wVaD8,student,1,2026-01-14T07:14:00Z,2025-12-05T07:14:00Z


In [92]:
# Task:
# Extract 15th record
df.iloc[15]

,15
user_id,16
username,user00016
email,user00016@univ.edu
password_hash,213fbf7dd2291db34d29062cc006984aabb98a815a20f9...
password_salt,Jahe84S5jIc1xLJj
role,student
mfa_enabled,1
last_login_utc,2026-01-14T08:33:00Z
created_utc,2025-12-10T08:33:00Z


In [105]:
# Task:
# Extract 10th record to 15th record``
df.iloc[10:16]

,user_id,username,email,password_hash,password_salt,role,mfa_enabled,last_login_utc,created_utc
10,11,user00011,user00011@example.com,2216a7d8714ce9cb290ac1c5af3028c16b33e2a59842cf...,dPI1af7W2pkAFEn3,student,0,2026-02-20T14:05:00Z,2026-01-20T14:05:00Z
11,12,user00012,user00012@corp.local,bf349ad9487b3ccd28b1d3d06cc6221c7b4396bb0ab0c0...,msn9dLVIdVuddLEG,student,0,2026-02-11T18:03:00Z,2026-01-06T18:03:00Z
12,13,user00013,user00013@corp.local,d8a1903e48f3eafb66e8064173a8b4d4d2dcff0cd4977c...,LMcNfAQLKHu7qnQT,staff,1,None,2026-01-26T12:56:00Z
13,14,user00014,user00014@example.com,046167458007fa4debdb82c3c997f28b50cf5cbadfac15...,eInGqi7w4e4pxskC,student,0,2025-12-19T05:21:00Z,2025-12-13T05:21:00Z
14,15,user00015,user00015@service.io,412dbefbe75a364345e656a0b8b3723db94dcbea5db90b...,jrsMnTvnRO2qGFq5,student,0,2026-01-19T16:58:00Z,2025-12-15T16:58:00Z
15,16,user00016,user00016@univ.edu,213fbf7dd2291db34d29062cc006984aabb98a815a20f9...,Jahe84S5jIc1xLJj,student,1,2026-01-14T08:33:00Z,2025-12-10T08:33:00Z


In [97]:
# Task:
# randomly select one record
df.sample()

,user_id,username,email,password_hash,password_salt,role,mfa_enabled,last_login_utc,created_utc
5090,5091,user05091,user05091@mail.test,77812019ed1f85a59d8f5c5f16ce7d60f4eeca591fb9ca...,engYXcgY06a2Vhda,auditor,0,2026-02-10T15:49:00Z,2026-01-11T15:49:00Z


In [98]:
# Task:
# pick randomly 10 records
df.sample(10)

,user_id,username,email,password_hash,password_salt,role,mfa_enabled,last_login_utc,created_utc
5210,5211,user05211,user05211@service.io,ccc3fd121f0870280de43dc092670ac1d270e1473682de...,5ziwfTleHP5djsau,student,1,2026-01-11T17:58:00Z,2026-01-03T17:58:00Z
2465,2466,user02466,user02466@example.com,21387b99832e2b9a3ce516a7fd0667d124647487b2fd33...,64FmRU6i096dnfQa,staff,0,2026-02-26T06:12:00Z,2026-01-16T06:12:00Z
3861,3862,user03862,user03862@service.io,4faaba289db4b154001ad295cbcc20d8c43f157bd741bc...,GPX4NnrVuFy81sKt,staff,0,2026-01-26T13:56:00Z,2025-12-08T13:56:00Z
660,661,user00661,user00661@example.com,b3498f19c2653f79dfd386f724606217f59a2da5d9ee16...,RW5TCaVRfuLX66xS,staff,0,2026-02-15T11:36:00Z,2026-01-15T11:36:00Z
1701,1702,user01702,user01702@univ.edu,18d26d987ede0f3aba35d6b66a8e9c8929409662c93443...,J5RJR4SAKGV137S5,student,0,2026-01-18T18:06:00Z,2026-01-09T18:06:00Z
5655,5656,user05656,user05656@corp.local,fa083d5dc29eb22f038b382e522cdb241ed6293c77f628...,RFJRGThNghvug9Mj,student,0,2026-01-14T16:48:00Z,2025-12-30T16:48:00Z
336,337,user00337,user00337@example.com,8f60fbfcd91d64b9145d5f25b95ed6cd489f3dff7edaa5...,9xk8KhaTHBX67D0R,staff,0,2026-02-16T06:12:00Z,2026-01-09T06:12:00Z
2695,2696,user02696,user02696@mail.test,c564fae50e153af0638952ce4d73505da3ac2e8aaafa2e...,Jgnfid9w4MpdJKHN,student,0,2026-01-29T02:29:00Z,2025-12-29T02:29:00Z
1547,1548,user01548,user01548@mail.test,92cf4b3adbdbd69f5c1ce856c76df645892d0872d5e980...,wL3VGXFLU6idBYjo,staff,0,2026-01-16T14:22:00Z,2025-12-10T14:22:00Z
2471,2472,user02472,user02472@example.com,9f70df58874195f9c5da14a080d61437e48e430f23d402...,wVm7Q6GOmR4tJ01P,student,0,2026-01-02T04:45:00Z,2025-12-31T04:45:00Z


In [106]:
# Task:
# Extract the column 'country'
df['email']

,email
0,user00001@example.com
1,user00002@corp.local
2,user00003@example.com
3,user00004@mail.test
4,user00005@corp.local
...,...
5995,user05996@example.com
5996,user05997@corp.local
5997,user05998@service.io
5998,user05999@corp.local


In [107]:
# Task:
# swaps the rows and columns
df.swapaxes(0, 1)
#df.T
#df.transpose()
# Note: it is used when we have more features

/tmp/ipython-input-1379086106.py:3: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  df.swapaxes(0, 1)


,0,1,2,3,4,5,6,7,8,9,...,5990,5991,5992,5993,5994,5995,5996,5997,5998,5999
user_id,1,2,3,4,5,6,7,8,9,10,...,5991,5992,5993,5994,5995,5996,5997,5998,5999,6000
username,user00001,user00002,user00003,user00004,user00005,user00006,user00007,user00008,user00009,user00010,...,user05991,user05992,user05993,user05994,user05995,user05996,user05997,user05998,user05999,user06000
email,user00001@example.com,user00002@corp.local,user00003@example.com,user00004@mail.test,user00005@corp.local,user00006@example.com,user00007@corp.local,user00008@service.io,user00009@example.com,user00010@example.com,...,user05991@example.com,user05992@corp.local,user05993@example.com,user05994@corp.local,user05995@mail.test,user05996@example.com,user05997@corp.local,user05998@service.io,user05999@corp.local,user06000@corp.local
password_hash,8d914e1cbb60b890c7399f2910cec96f8a5a8271363223...,581e669218963490d73113876dde788b242cedd3bf6cfe...,53a95abbb5fc355643afa051e051096dcccbcdad3332c5...,8330d1676585fc69ec535bed89b7975db8b0a23501a8ff...,f2889184f365ce51dd86bcdeaa4b03f74cd938666af404...,8fc5e174e1ba391a10e72f60c2ed6c9fc4d8b28b8c6800...,5ffe482fbdddeef2a9c45ca7e97c6918a5390ea1154fa6...,44cef8e1ebedd1ced27557fd814c5cfd14d996b367273b...,d647ee8645b5faf2aea068260be22c0a4559d90980e118...,85ad4d23a799be4ac733105fa0777019d5cce3dbaa403a...,...,373ee42c1d25579da009006d5cb26474f4d070d25b545d...,5df2500f1ad01293301895dd26174c3f57d1cd03c81120...,69a441e681264a10318f46518b504c122540652f51822c...,228073cab25cb05d2177a26b59856ba91539c6bda64754...,b2ff31419a2969d041931e1d18992093ac4d4260b78f14...,b5eeefc9a6aa2ba2126f999db37b163ecd93919620f3f1...,7478fedc0c6703ab0313b84a47a1cee2a4dcbd89e595fe...,e6ca07d767ad01a8ab27a52cbaf04e85a7e31f39db9367...,1b92217c412b25b5a6443ff47c70dc43c3daf94b2918ec...,79ba122c6001903c87a94a0406748391f059d9261a0d24...
password_salt,bVrpoiVgRV5IfLBc,SBvrjn9Wvgfygw2w,cQoXsf2o3gyrDO1x,7OSJoRu1XXdo0cZu,pVJIqVLB5LzxoiGF,RUhR4IWrXPvhsBkD,xWkI9X7H6aMuFbh7,XiiQE8JkqH3MB9n7,LJoLoaeTOdoe5c3v,gQBwBAD3UdRPPgdz,...,ypU8tz9OVmMKHUOh,e33rXnzuV7dpxa2s,6DnEvqDBZPXbRoIo,IxYoPicJgifpf2w8,fBDMbGSQux6wVaD8,1oU49MRWypWAO7hb,OnV3joXTYF6g8JuB,wbL15YRXMMA0sj8H,Jf0ZzGdH9pv6uTQE,v5F33Jia3jFIVaOK
role,student,student,student,student,student,staff,student,staff,student,student,...,student,student,student,student,student,student,student,student,student,staff
mfa_enabled,1,0,0,0,0,1,0,0,0,0,...,0,0,1,1,1,0,0,1,1,0
last_login_utc,2026-01-23T15:16:00Z,2026-02-16T22:00:00Z,2026-01-16T06:47:00Z,2026-02-13T12:19:00Z,2026-02-16T14:28:00Z,2026-02-09T03:48:00Z,None,2026-01-13T13:33:00Z,2026-01-25T01:13:00Z,2026-01-11T20:40:00Z,...,2026-02-22T16:44:00Z,2026-02-09T19:58:00Z,2026-03-01T05:51:00Z,2025-12-16T11:57:00Z,2026-01-14T07:14:00Z,None,2026-02-11T04:47:00Z,2026-02-07T19:15:00Z,2026-01-19T19:58:00Z,2026-02-07T23:02:00Z
created_utc,2026-01-23T15:16:00Z,2026-01-02T22:00:00Z,2025-12-23T06:47:00Z,2026-01-28T12:19:00Z,2026-01-12T14:28:00Z,2026-01-28T03:48:00Z,2025-12-07T07:11:00Z,2025-12-23T13:33:00Z,2026-01-13T01:13:00Z,2025-12-07T20:40:00Z,...,2026-01-23T16:44:00Z,2026-01-07T19:58:00Z,2026-01-23T05:51:00Z,2025-12-09T11:57:00Z,2025-12-05T07:14:00Z,2025-12-20T06:43:00Z,2026-01-29T04:47:00Z,2026-01-23T19:15:00Z,2025-12-20T19:58:00Z,2025-12-21T23:02:00Z


In [101]:
# Task: list all the column name
df.columns


Index(['user_id', 'username', 'email', 'password_hash', 'password_salt',
       'role', 'mfa_enabled', 'last_login_utc', 'created_utc'],
      dtype='object')

In [102]:
# list all the datatypes
df.dtypes

,0
user_id,object
username,object
email,object
password_hash,object
password_salt,object
role,object
mfa_enabled,object
last_login_utc,object
created_utc,object


In [103]:
df.sample()

,user_id,username,email,password_hash,password_salt,role,mfa_enabled,last_login_utc,created_utc
3326,3327,user03327,user03327@mail.test,3902e2c01b03e4a5dacbac3bb8445a4742a94cfa424bc6...,uxj8McOamwN3kOb9,student,0,2026-02-22T19:52:00Z,2026-01-07T19:52:00Z


Pandas Datatypes
    - Numeric
          - int
          - float

    - String
       - fixed values
          - category
       - not fixed values
          - object

    - Boolean
        - bool

    - Datetime
          - datetime
          - timedelta

In [ ]:
# change a column’s data type to the corresponding (appropriate) datatype
# step : Inspect -->   Decide --> Convert  --> Verify
# user_id

In [109]:
df.columns

Index(['user_id', 'username', 'email', 'password_hash', 'password_salt',
       'role', 'mfa_enabled', 'last_login_utc', 'created_utc'],
      dtype='object')

In [110]:
# change user_id to int
df['user_id'] = df['user_id'].astype('int')

In [111]:
df.dtypes

,0
user_id,int64
username,object
email,object
password_hash,object
password_salt,object
role,object
mfa_enabled,object
last_login_utc,object
created_utc,object


In [135]:
# role
df['role'] = df['role'].astype('category')

In [136]:
df.dtypes

,0
user_id,int64
username,object
email,object
password_hash,object
password_salt,object
role,category
mfa_enabled,bool
last_login_utc,"datetime64[ns, UTC]"
created_utc,"datetime64[ns, UTC]"


In [143]:
df["mfa_enabled"] = df["mfa_enabled"].astype(bool)
df["mfa_enabled"]

,mfa_enabled
0,True
1,True
2,True
3,True
4,True
...,...
5995,True
5996,True
5997,True
5998,True


In [147]:
#mfa_enabled
df["mfa_enabled"] = df["mfa_enabled"].map({"1": True, "0": False})

In [149]:
# if it mixed pattern
df["mfa_enabled"] = df["mfa_enabled"].str.lower().map({
    "1": True, "0": False,
    "yes": True, "no": False,
    "true": True, "false": False
})
df["mfa_enabled"]

,mfa_enabled
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
5995,NaN
5996,NaN
5997,NaN
5998,NaN


In [117]:
# last_login_utc
df["last_login_utc"] = pd.to_datetime(df["last_login_utc"])

In [118]:
# created_utc
df["created_utc"] = pd.to_datetime(df["created_utc"])

In [119]:
#Check whether username and email are unique.
df["username"].is_unique

True

In [121]:
# To check only particular column
df["email"].is_unique

True

In [123]:
#Identify which column behaves like an identifier (very high unique count).
df.nunique()

,0
user_id,6000
username,6000
email,6000
password_hash,6000
password_salt,6000
role,4
mfa_enabled,2
last_login_utc,5269
created_utc,5783


In [124]:
#Count how many records are fully duplicated rows
df.duplicated().sum()

np.int64(0)

In [129]:
#Display only the username column
df["username"]


,username
0,user00001
1,user00002
2,user00003
3,user00004
4,user00005
...,...
5995,user05996
5996,user05997
5997,user05998
5998,user05999


In [130]:
#Display the username and email columns together.
df[["username", "email"]]

,username,email
0,user00001,user00001@example.com
1,user00002,user00002@corp.local
2,user00003,user00003@example.com
3,user00004,user00004@mail.test
4,user00005,user00005@corp.local
...,...,...
5995,user05996,user05996@example.com
5996,user05997,user05997@corp.local
5997,user05998,user05998@service.io
5998,user05999,user05999@corp.local


In [ ]:
#What is the difference between:

df_users["email"]

df_users[["email"]]

In [131]:
#Access the first row using positional indexing.
df.iloc[0]

,0
user_id,1
username,user00001
email,user00001@example.com
password_hash,8d914e1cbb60b890c7399f2910cec96f8a5a8271363223...
password_salt,bVrpoiVgRV5IfLBc
role,student
mfa_enabled,True
last_login_utc,2026-01-23 15:16:00+00:00
created_utc,2026-01-23 15:16:00+00:00


In [133]:
#Access the first 5 rows using slicing.
df[:5]

,user_id,username,email,password_hash,password_salt,role,mfa_enabled,last_login_utc,created_utc
0,1,user00001,user00001@example.com,8d914e1cbb60b890c7399f2910cec96f8a5a8271363223...,bVrpoiVgRV5IfLBc,student,True,2026-01-23 15:16:00+00:00,2026-01-23 15:16:00+00:00
1,2,user00002,user00002@corp.local,581e669218963490d73113876dde788b242cedd3bf6cfe...,SBvrjn9Wvgfygw2w,student,False,2026-02-16 22:00:00+00:00,2026-01-02 22:00:00+00:00
2,3,user00003,user00003@example.com,53a95abbb5fc355643afa051e051096dcccbcdad3332c5...,cQoXsf2o3gyrDO1x,student,False,2026-01-16 06:47:00+00:00,2025-12-23 06:47:00+00:00
3,4,user00004,user00004@mail.test,8330d1676585fc69ec535bed89b7975db8b0a23501a8ff...,7OSJoRu1XXdo0cZu,student,False,2026-02-13 12:19:00+00:00,2026-01-28 12:19:00+00:00
4,5,user00005,user00005@corp.local,f2889184f365ce51dd86bcdeaa4b03f74cd938666af404...,pVJIqVLB5LzxoiGF,student,False,2026-02-16 14:28:00+00:00,2026-01-12 14:28:00+00:00


In [134]:
#Access the 10th row (by position).
df.iloc[10]

,10
user_id,11
username,user00011
email,user00011@example.com
password_hash,2216a7d8714ce9cb290ac1c5af3028c16b33e2a59842cf...
password_salt,dPI1af7W2pkAFEn3
role,student
mfa_enabled,False
last_login_utc,2026-02-20 14:05:00+00:00
created_utc,2026-01-20 14:05:00+00:00


In [150]:
#Access the value of email in the 5th row using positional indexing.
df.iloc[5]["email"]

'user00006@example.com'

In [151]:
#Access the record corresponding to user_id = 100.
df[df["user_id"] == 100]


,user_id,username,email,password_hash,password_salt,role,mfa_enabled,last_login_utc,created_utc
99,100,user00100,user00100@univ.edu,29e3bc5648e8a75eeebf9652fc2d102746ce96c2005f57...,DKdJDRZtUbzq0aVn,student,NaN,2025-12-09 19:17:00+00:00,2025-12-06 19:17:00+00:00


In [152]:
#Access only the email of user_id = 100.
df[df["user_id"] == 100]["email"]

,email
99,user00100@univ.edu


In [153]:
#Access all records where role is admin.
df[df["role"] == "admin"]

,user_id,username,email,password_hash,password_salt,role,mfa_enabled,last_login_utc,created_utc
20,21,user00021,user00021@univ.edu,c1288b40d36e54c7b8b0b965a2cebc285e7fdc49b32bae...,YLMPuDCCRnGEY59Y,admin,NaN,2026-01-10 06:07:00+00:00,2025-12-29 06:07:00+00:00
27,28,user00028,user00028@corp.local,351a994c3fa9d9d639061cbc251de2c7987727a54f076a...,m1bNjpiEQhK8nDSq,admin,NaN,NaT,2025-12-03 08:05:00+00:00
88,89,user00089,user00089@service.io,4baba6c92e01a5a95a68582d36db2050a726d3ecbe9657...,IFTKtE0bxvRhALtY,admin,NaN,2026-01-27 17:04:00+00:00,2025-12-21 17:04:00+00:00
142,143,user00143,user00143@corp.local,87d27eb1865a0afe2cc8ebfd10ee603dbe5135abc98d45...,a7AboIwO1S4a8vaW,admin,NaN,2025-12-18 23:01:00+00:00,2025-12-13 23:01:00+00:00
151,152,user00152,user00152@univ.edu,c9f1f529cda5a06940f137807b786b97f71dd9b4fe4c00...,av6NmZivTl6Z23ud,admin,NaN,2026-01-27 00:11:00+00:00,2026-01-06 00:11:00+00:00
...,...,...,...,...,...,...,...,...,...
5793,5794,user05794,user05794@service.io,83053215feec3408ecdc1fd70acf9b9c8e067329d6e6f1...,lIzfqLkLeP73XC9w,admin,NaN,2026-01-25 11:57:00+00:00,2025-12-06 11:57:00+00:00
5885,5886,user05886,user05886@corp.local,daa66b98daa0005b1732888b2c75451d0b0beafb107ebd...,FuwqxIxMk2qEX4AH,admin,NaN,2025-12-18 07:18:00+00:00,2025-12-09 07:18:00+00:00
5910,5911,user05911,user05911@example.com,3c834787ca48bf9f50ebbf81fd2156f199a8db66ea0c60...,ao5JHMbdOjXgMF2F,admin,NaN,NaT,2026-01-17 12:16:00+00:00
5917,5918,user05918,user05918@example.com,a2385c5921322f0d58c4ba7ed46f5b6629250d825ba8b6...,2oCzak9lgl4UTrbw,admin,NaN,2026-02-16 16:25:00+00:00,2026-01-26 16:25:00+00:00


In [ ]:
#Access all records where mfa_enabled is "0".

In [ ]:
# Access records where role is staff AND mfa_enabled is "0".



In [ ]:
#Access records where role is either admin OR staff.

In [ ]:
#Access multiple users using index labels: user_id = 10, 20, 30.

In [ ]:
#From users with mfa_enabled = "0", display only: user_id , username , email

In [ ]:
#From users whose role is admin, display only: username , created_utc , last_login_utc

In [ ]:
#Access users whose role is admin using query().


In [ ]:
#Access users whose role is staff and MFA is disabled using query().